1. Download the dataset

In [11]:
from pathlib import Path
from torch.utils.data import DataLoader, Subset
import torch.cuda
import requests
import zipfile

ROOT = Path("./data")
if not ROOT.exists():
    ROOT.mkdir()
    
CUDA_AVAILABLE = torch.cuda.is_available()

DATASET_PATH = ROOT / "vggface2"
if not DATASET_PATH.exists():
    DATASET_PATH.mkdir()
    with requests.get("https://www.kaggle.com/api/v1/datasets/download/hearfool/vggface2", stream=True) as r:
        r.raise_for_status()
        tmp = DATASET_PATH / "tmp"
        with tmp.open("wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

    with zipfile.ZipFile(str(tmp)) as zip_ref:
        zip_ref.extractall(str(DATASET_PATH))
    tmp.unlink()
        

2. Compute embbeding vector for each image on the dataset.

In [ ]:
from time import monotonic
from torchvision.datasets import ImageFolder
from torchvision import transforms
from IPython.display import display
import cv2
import matplotlib.pyplot as plt
import numpy as np
from insightface.app import FaceAnalysis
from torch import nn

app = FaceAnalysis(root=str(ROOT / "insightface"), name='buffalo_l', providers=['CPUExecutionProvider' if not CUDA_AVAILABLE else 'CUDAExecutionProvider'])  # Use 'CUDAExecutionProvider' for GPU
app.prepare(ctx_id=(-1 if not CUDA_AVAILABLE else 0))  # ctx_id=-1 for CPU, 0 for GPU

pil_transform = transforms.ToPILImage()
t = transforms.Compose([
    transforms.Resize((256, 256)),
])

def from_tensor_to_cv2_like(tensor):
    return cv2.cvtColor(np.array(tensor), cv2.COLOR_RGB2BGR)

def show_images(images, titles):
    plt.figure(figsize=(10, 8))
    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(2, 2, i + 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
        plt.title(title)
        plt.axis("off")

    plt.tight_layout()
    plt.show()

vggface2 = ImageFolder(DATASET_PATH / "val", transform=t)
subset = Subset(vggface2, range(100))
loader = DataLoader(subset, batch_size=10, shuffle=True)



begin = monotonic()
embedding2 = app.get(cv2.imread("./data/vggface2/val/n000001/0001_01.jpg"))[0].embedding
print(f"TOOK: {embedding2}")


#def compare_faces(emb1, emb2, threshold=0.65): # Adjust this threshold according to your usecase.
#    """Compare two embeddings using cosine similarity"""
#    similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
#    return similarity, similarity > threshold
#
#print(compare_faces(embedding2, embedding3))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: data/insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: data/insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: data/insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: data/insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: data/insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
TOOK: 0.60987485199802